In [1]:
head = 'l30_r15'
directory = '../data/'
model_fname ='../models/Logistic + L2 (C=1.00) norm l30_r15.pkl'


import pickle    
from py.utils import load_data

X, y, x_words, vocabs = load_data(head, directory)
with open(model_fname, 'rb') as f:
    classifier = pickle.load(f)

x shape = (15105, 2770)
y shape = (15105,)
# features = 2770
# L words = 15105


In [2]:
coefficient = {vocabs[j]:coef for j, coef in enumerate(classifier.coef_[0])}

In [3]:
list(coefficient.items())[:5]

[('버리자', -0.012245416496418466),
 ('만은', 0.043057437931686038),
 ('어주던', -0.00075593073444216836),
 ('면서부터는', -0.001841008788851098),
 ('이다라고', 0.0010159583420968812)]

In [4]:
import sys
from soynlp.utils import get_process_memory
from soynlp.hangle import normalize
import numpy as np

In [5]:
# with open(sentence_fname, encoding='utf-8') as f:
#     with open('../data/sentences_onlyhangle.txt', 'w', encoding='utf-8') as fo:        
#         for sent in f:
#             fo.write('%s\n' % normalize(sent.strip()))
# print('done')

In [6]:
class TrainedNounExtractor:
    def __init__(self, coefficient, max_length=8):
        self._coef = coefficient
        self.lmax = max_length
        
    def extract(self, sents, min_count=10, min_noun_score=0.1):
        self.lrgraph, self.lset, self.rset = self._build_lrgraph(sents, min_count)
        self.lentropy, self.rentropy, self.lav, self.rav = self._branching_entropy(self.lrgraph)
        scores = self._compute_noun_score(self.lrgraph)
        return scores, self.lrgraph
        scores = self._postprocessing(lrgraph, scores)

    def _build_lrgraph(self, sents, min_count, pruning_min_count=2):
        lset = {}
        rset = {}
        for n_sent, sent in enumerate(sents):
            for eojeol in sent.split():
                for e in range(1, min(len(eojeol), self.lmax)+1):
                    l = eojeol[:e]
                    r = eojeol[e:]
                    lset[l] = lset.get(l, 0) + 1
                    rset[r] = rset.get(r, 0) + 1
            if n_sent % 1000 == 999:
                args = (n_sent+1, len(lset), len(rset), get_process_memory())
                sys.stdout.write('\rscaning vocabulary ... %d sents #(l= %d, r= %d), mem= %.3f Gb' % args)
            if n_sent % 500000 == 499999:
                lset = {l:f for l,f in lset.items() if f >= pruning_min_count}
                rset = {l:f for l,f in rset.items() if f >= pruning_min_count}
        lset = {l:f for l,f in lset.items() if f >= min_count}
        rset = {l:f for l,f in rset.items() if f >= min_count}
        
        n_sents = n_sent
        
        lrgraph = {}
        for n_sent, sent in enumerate(sents):
            for eojeol in sent.split():
                for e in range(1, min(len(eojeol), self.lmax)+1):
                    l = eojeol[:e]
                    r = eojeol[e:]
                    if not (l in lset) or not (r in rset):
                        continue
                    rdict = lrgraph.get(l, {})
                    rdict[r] = rdict.get(r, 0) + 1
                    lrgraph[l] = rdict            
            if n_sent % 1000 == 999:
                args = (100*(n_sent+1)/n_sents, '%', n_sent+1, n_sents, get_process_memory())
                sys.stdout.write('\rbuilding lrgraph ... (%.3f %s, %d in %d), mem= %.3f Gb' % args)
        args = (len(lset), len(rset), sum((len(rdict) for rdict in lrgraph.values())), get_process_memory())
        print('\rlrgraph has been built. (#L= %d, #R= %d, #E=%d), mem= %.3f Gb' % args)
        return lrgraph, lset, rset
    
    def _branching_entropy(self, lrgraph):
        from collections import defaultdict
        def entropy_and_accessorvariety(d):
            sum_ = sum(d.values())
            if sum_ == 0: return (0, 0)
            return (-1 * sum((v/sum_) * np.log(v/sum_) for v in d.values()), len(d))
        def branch_map(d, get_branch):
            b = defaultdict(lambda: 0)
            for ext,f in d.items():
                if ext != '':
                    b[get_branch(ext)] += f
            return b
        def all_entropy(graph, get_branch=lambda x:x[0]):
            return {w:entropy_and_accessorvariety(branch_map(d, get_branch)) for w, d in graph.items()}
        def to_rlgraph(lrgraph):
            rlgraph = defaultdict(lambda: defaultdict(lambda: 0))
            for l, rdict in lrgraph.items():
                for r, f in rdict.items():
                    if r == '': continue
                    rlgraph[r][l] += f
            return {r:dict(ldict) for r, ldict in rlgraph.items()}
        print('compute branching entropy ...', end='')
        lvalues = all_entropy(lrgraph)
        rvalues = all_entropy(to_rlgraph(lrgraph), get_branch=lambda x:x[-1])
        lentropy = {w:v[0] for w,v in lvalues.items()}
        rentropy = {w:v[0] for w,v in rvalues.items()}
        lav = {w:v[1] for w,v in lvalues.items()}
        rav = {w:v[1] for w,v in lvalues.items()}
        print(' done')
        return lentropy, rentropy, lav, rav
        
    def _compute_noun_score(self, lrgraph):
        from collections import namedtuple
        Score = namedtuple('Score', 'score frequency be av p_feature p_eojeol')
        scores = {}
        n = len(lrgraph)
        for i, (l, rdict) in enumerate(lrgraph.items()):
            rdict_ = {r:f for r,f in rdict.items() if r in self._coef}
            rsum = sum((f for r,f in rdict.items() if r != ''))
            frequency = self.lset.get(l, 0)
            feature_fraction = sum(rdict_.values()) / rsum if rsum > 0 else 0
            eojeol_fraction = rdict.get('', 0) / frequency
            if not rdict_:
                score = 0
            else:
                score = sum(f*self._coef[r] for r, f in rdict_.items()) / sum(rdict_.values())
            scores[l] = Score(score, frequency, self.lentropy.get(l, 0), self.lav.get(l,0), feature_fraction, eojeol_fraction)
            if (i+1) % 1000 == 0:
                args = (100*(i+1)/n, '%', i+1, n)
                sys.stdout.write('\rcompute noun score ... (%.3f %s, %d in %d)' % args)
        print('\rcomputing noun score has been done.')
        return scores
#         return sorted(scores.items(), key=lambda x:x[1].score, reverse=True)
        
    def _postprocessing(self, lrgraph, scores):
        print('hi')

In [7]:
from config import sentence_fname
class Sentences:
    def __init__(self, fname):
        self.fname = fname
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                yield doc.strip()

sentences = Sentences('../data/sentences_onlyhangle.txt')
noun_extractor = TrainedNounExtractor(coefficient)
scores, lrgraph = noun_extractor.extract(sentences)

lset, rset = noun_extractor.lset, noun_extractor.rset

lrgraph has been built. (#L= 173620, #R= 71297, #E=2022472), mem= 0.442 Gb
compute branching entropy ... done
computing noun score has been done.


In [8]:
def to_rlgraph(lrgraph):
    from collections import defaultdict
    rlgraph = defaultdict(lambda: defaultdict(lambda: 0))
    for l, rdict in lrgraph.items():
        for r, f in rdict.items():
            if r == '': continue
            rlgraph[r][l] += f
    return {r:dict(ldict) for r, ldict in rlgraph.items()}

rlgraph = to_rlgraph(lrgraph)

## Preprocessor, Postprocessing의 필요성

- lrgraph를 만들 때 한글 외의 기호를 삭제한 어절로 이뤄져야 함 
- N = Nsub + J problem
    - 중세보편주 + {의, 의의}
- [N+Jsub] + J problem
    - 대학생과 + {'', 의}
    - {의, 과의} 모두 조사
- 1음절 명사들은 제외하였다. 필요하면 그 때 추가하도록. 아래는 0.5 를 넘은 1음절 명사들이다 

    질 럼 듀 균 흑 첨 응 쌘 초 삥 휭 릴 변 문 핸 윈 심 팅 뉴 를 맏 갱 흥 국 결 잣 휴 콤 도 닉 퓌 쫌 넷 놋 깡 앨 캡 불 횡 완 샤 원 삽 당 비 총 서 시 튜 골 ㅅ 판 범 옥 꽤 껀 빈 란 장 셰 징 산 송 쎈 덴 쑥 쉼 냇 펑 택 밸 군 외 표 멍 볼 카 렌 평 딱 쟁 헛 웹 멧 귄 띵 탈 망 번 케 절 퍽 약 갯 깃 그 셋 피 캠 뿐 턴 롤 흔 겹 빅 철 간 촌 령 측 꽝 겸 논 암 뚝 욱 얀 뭣 취 껄 칭 때 면 쇄 쇳 안 잠 푼 종 말 쌈 차 성 침 길 의 칠 호 향 전 형 킴 흉 패 둘 엠 끝 양 넙 람 옴 악 통 싯 팝 수 좀 새 배 눔 쿵 채 회 똔 괄 림 윷 벱 발 벨 랑 날 일 분 쯔 찡 읍 콘 픽 편 헉 극 셈 햄 관 제 팽 또 팬 권 쾅 땐 흠 너 격 탕 뻑 훤 목 휠 병 휑 즤 후 김 톱 잔 숀 쿨 욕 탭 묘 젤 방 뻔 잼 것 누 맛 급 닌 갓 겉 척 끗 링 화 탓 멜 못 잭 내 곡 답 강 족 걔 법 뻥 킬 귤 년 뽁 중 애 득 책 셸 루 빤 죄 곰 똘 멱 꾀 뜻 궤 쟤 늬 록 놈 쿡 룰 폼 폭 옻 션 펠 플 윌 캔 브 갠 뜸 칩 홑 덕 앞 월 뚱 앳 섬 왕 힘 숄 론 글 돛 밥 맘 똥 키 대 식 위 맥 틱 점 떡 룸 빽 값 꼿 빛 폴 늠 색 쇠 움 봄 집 뼘 바 쯤 딸 씽 꽃 뭔 곳 첩 손 락 딕 껌 빵 톰 뿔 잽 껩 술 짬 획 숍 녘 냥 펙 앰 꿈 탑 숯 벽 씩 량 층 갭 잎 펌 겐 북 착 딘 팩 힐 펜 리 벼 곽 삯 프 티 핵 댁 홈 옷 칡 별 꿩 뇌 춤 예 턱 밤 뽕 쪽 짐 개 씨 삔 굿 꼴 콩 붐 팥 솜 샬 넋 칸 밭 쉿 휙 꾹 돈 칼 팀 니 듯 빔 샘 눈 랩 몫 펄 컵 둑 툴 틈 꿀 뱀 컷 쉘 씬 흙 팁 큐 훅 앎 늪 숲 속 흐 뫼 샅 돔 뒤 뼈 쌀 닭 코 곁 뺨 쨍 볏 샷 뽈 뮤 쟝 밑 괌 홀 괘 롬 짝 맴 명 빰 녜 싹 팸 퀭 험 땀 닢 덤 볕 슛 뜰 몸 땅 즙 킥 홉 닻 뭍 립 밖 솥 쑈 퀸 텔 덫 옆 쇼 퉤 쌍 댐 룩 으 램 땜 닐 귀 콕 톤 첼 등 숱 텀 샨 쿤 섶 력 꾐 윙 씰 훠 쩜 엥 콰 캉 팰 삶 님 킹 네 므 류 괜 톨 혀 홰 웜 녀 갸

In [9]:
# useful function
def pretty(namedtuple_instance, end='\n'):
    print('%s(%s)' % (namedtuple_instance.__class__.__name__, ', '.join(['%s=%d' % (field, value) if (type(value) == int or type(value) == np.int) else '%s=%.3f' % (field, value)  for field, value in namedtuple_instance._asdict().items()])),end=end )

def pretty_list(l):
    print('[', end='')
    for w, nt in l[:10]:
        print('(%s,'%w, end=' ')
        pretty(nt, end='),\n')
    print(']' if len(l) <= 10 else '... ]')

In [10]:
lrgraph['중세보편주']

{'의': 6, '의를': 7, '의의': 2}

In [11]:
lrgraph['대학생과']

{'': 25, '의': 1}

In [12]:
sorted(lrgraph['대학생'].items(), key=lambda x:x[1], reverse=True)[:10]

[('', 206),
 ('들이', 76),
 ('이', 76),
 ('들의', 68),
 ('들은', 39),
 ('을', 29),
 ('의', 26),
 ('과', 25),
 ('들', 24),
 ('들을', 23)]

In [13]:
pretty_list(list(filter(lambda x:x[1].frequency > 100 and len(x[0]) > 1, scores.items()))[:3])

[(배웠다, Score(score=-2.745, frequency=185, be=1.096, av=6, p_feature=0.979, p_eojeol=0.741)),
(말하고, Score(score=-0.512, frequency=1293, be=1.085, av=8, p_feature=0.986, p_eojeol=0.889)),
(변함없, Score(score=1.389, frequency=107, be=1.203, av=8, p_feature=1.000, p_eojeol=0.000)),
]


In [14]:
pretty_list(list(scores.items())[:3])

[(안색을, Score(score=0, frequency=19, be=0, av=0, p_feature=0, p_eojeol=1.000)),
(효과라, Score(score=-1.945, frequency=11, be=0.530, av=2, p_feature=1.000, p_eojeol=0.182)),
(이거예, Score(score=-0.004, frequency=30, be=-0.000, av=1, p_feature=1.000, p_eojeol=0.000)),
]


## Dev: post-processing

### branching entropy는 좀 구해볼까? 
    
    L, right-side 
    R, left-side
    
    ## Complete
    
    >>> for w in ['대학', '대학생', '대학생과']:
    >>>     print(w, '%.3f' % lentropy.get(w, 0))

    대학 2.832
    대학생 1.721
    대학생과 0.000

In [15]:
min_noun_score = 0.5

In [16]:
score_dict = dict(scores)
s_ = score_dict['떡볶']
s = score_dict['떡볶이']
pretty(s_)
pretty(s)


Score(score=2.019, frequency=77, be=0.406, av=2, p_feature=0.620, p_eojeol=0.000)
Score(score=3.230, frequency=67, be=1.944, av=13, p_feature=0.925, p_eojeol=0.388)


### Nsub + J: 떡볶 + 이

    f(떡볶) ~= f(떡볶이): drop-rate, branching entropy로 하자
    (떡볶이 in Noun) and (이 in Josa)
    주로 한글자 짜리 조사가 문제 

In [17]:
josa = sorted(filter(lambda x:x[1] > -0.1, coefficient.items()), key=lambda x:x[1], reverse=True)
josa_dict = dict(josa)
len(josa_dict)

2573

In [18]:
' '.join(tuple(zip(*(filter(lambda x:x[1] > 0.1, josa))))[0])

'의 가 를 한 인 이 에 으로 로 을 된 하는 할 하고 에서 과 하게 에게 와 되는 만 까지 되어 부터 에는 해 처럼 하여 이나 로운 도 이고 해야 스럽게 되고 치 하지 해서 될 라고 롭게 없이 롭지 하기 이지 에서는 마다 이라도 시키는 든지 인지 같이 로는 보다 되지 하면 으로는 라는 돼 스런 정책 스러운 나 한테 하면서 이라고 일 시켜 여서 라도 이라는 하며 엔 였고 니까 시키고 서도 되어야 이면 에게는 시킬 까지나 이나마 하던 한다는 시킨 에서도 이던 서만 란 인데 만한 이야 이지만 되기 롭고 하거나 니 되면 랑 은 만의 시키기 에다 된다는 보다는'

In [19]:
postnoun = set('가감계과권꾼네님대댁당들력로론류률상생성시식용율의인이일재적제중째쯤층치판풍형화')
def find_composed_josa(josa):
    offset = set('계권꾼네님대력론류상생성시식용율적째쯤층치풍형화')
    return {j for j in josa if ((len(j) > 1) and (j[0] in josa) and not (j[0] in offset) and (j[1:] in josa))}

composed_josa = find_composed_josa(tuple(zip(*josa))[0])
jsub_set = {j[0] for j in composed_josa}

명사 뒤에 결합하여 의미를 확장하는 의존명사 (postnoun)이면서 조사에서는 잘 이용되지 않는 1음절 단어들을 offset으로 등록하였다. 

의존명사는 명사를 추출하는 질 좋은 R set feature이다. '나무 + 꾼' 처럼 '나무'가 명사라는 걸 잘 알려주기 때문이다. 하지만 실제로는 '나무꾼'을 단일 명사로 추출해야 한다. 

그 외에는 [-가, -이, -로, ...]의 경우 조사일 수도 있고, 다른 조사의 left-side 일수도 있다. 해당하는 조사들과 각각의 coefficient를 나열하면 아래와 같다. 

    >>> sorted({(j[0], '['+', '.join(tuple(zip(*sorted({'-'+j_[1:]:coef for j_,coef in coefficient.items() if len(j_) > 1 and j[0] == j_[0]}.items(), key=lambda x:x[1], reverse=True)))[0][:5])+']', '%.3f'%coefficient[j[0]]) for j in composed_josa}, key=lambda x:x[2], reverse=True)

    [('가', '[-에, -가, -의, -에서, -로]', '4.399'),
     ('한', '[-다, -테, -다는, -다고, -다면]', '3.766'),
     ('에', '[-서, -게, -는, -서는, -게는]', '3.551'),
     ('로', '[-운, -는, -서, -서는, -군]', '3.540'),
     ('이', '[-다, -나, -었다, -지, -야]', '3.281'),
     ('할', '[-만한, -지, -지도, -까봐, -수록]', '2.032'),
     ('인', '[-지, -가, -데, -들, -데요]', '1.411'),
     ('와', '[-의, -는, -도, -같이, -같은]', '1.398'),
     ('과', '[-는, -도, -의]', '1.137'),
     ('해', '[-서, -야, -진, -도, -서는]', '1.047'),
     ('만', '[-한, -의, -큼, -을, -은]', '0.791'),
     ('나', '[-마, -요, -면, -의, -구]', '0.723'),
     ('제', '[-를, -는, -의, -가, -와]', '0.630'),
     ('들', '[-은, -이, -의, -을, -에게]', '0.592'),
     ('씩', '[-이나, -은, -을, -만, -이라도]', '0.589'),
     ('도', '[-로, -가도, -다, -록]', '0.530'),
     ('당', '[-하고, -했다, -할, -하게, -하거나]', '0.257'),
     ('엔', '[-가, -지]', '0.240'),
     ('률', '[-이, -을, -은, -도, -과]', '0.171'),
     ('일', '[-까, -지라도, -지, -까요, -수록]', '0.154'),
     ('냐', '[-며, -는, -면은, -면, -라고]', '0.132'),
     ('랑', '[-은, -도]', '0.095'),
     ('케', '[-하는]', '0.065'),
     ('뿐', '[-인, -만, -이었다, -이다, -인데]', '0.031'),
     ('별', '[-로, -로는]', '0.025'),
     ('댁', '[-은, -이]', '0.018'),
     ('함', '[-으로써, -을, -이, -에, -과]', '0.018'),
     ('키', '[-로]', '0.012'),
     ('께', '[-서, -서는, -서도, -선, -로]', '0.007'),
     ('경', '[-에는, -에, -제, -제적]', '0.005'),
     ('됨', '[-에, -으로써, -을, -이, -과]', '0.004'),
     ('감', '[-이, -을]', '0.003'),
     ('덜', '[-이]', '0.001'),
     ('미', '[-국, -일]', '0.001'),
     ('임', '[-을, -에도, -이, -은, -에]', '0.000'),
     ('하', '[-고, -는, -게, -여, -지]', '0.000')]
     
그러므로 composed_josa의 첫글자들이 명사 후보의 맨 뒤에 붙어있으면 의심을 해보아야 한다

    >>> jsub_set = {j[0] for j in composed_josa}

    '가 감 경 과 께 나 냐 당 댁 덜 도 됨 들 랑 로 률 만 미 별 뿐 씩 에 엔 와 이 인 일 임 제 케 키 하 한 할 함 해'


In [20]:
def is_NsubJ(s_, s, bes_=0.5, f_fraction=0.5, eojeol_fraction_s_=0.1):
    return (s_.be < bes_) and (s.frequency / s_.frequency > f_fraction) and (s_.p_eojeol < eojeol_fraction_s_)

pretty(score_dict['떡볶'])
pretty(score_dict['떡볶이'])

Score(score=2.019, frequency=77, be=0.406, av=2, p_feature=0.620, p_eojeol=0.000)
Score(score=3.230, frequency=67, be=1.944, av=13, p_feature=0.925, p_eojeol=0.388)


### [N + Jsub] + J: 대학생과 + 의

    f(대학생) >> f(대학생과)
    Right side branching entropy(대학생) ~= high
    (과의 in Josa) and (대학생 in Noun)

In [21]:
pretty(score_dict['대학'])
pretty(score_dict['대학생'])
pretty(score_dict['대학생과'])

Score(score=3.078, frequency=8678, be=2.832, av=93, p_feature=0.601, p_eojeol=0.232)
Score(score=1.992, frequency=852, be=1.925, av=32, p_feature=0.553, p_eojeol=0.242)
Score(score=6.485, frequency=26, be=-0.000, av=1, p_feature=1.000, p_eojeol=0.962)


In [22]:
def is_NJsub(s_, s, bes_=0.5, bes=0.5, eojeol_fraction_s=0.5, f_fraction=0.3):
    return (s_.be > bes_) and (s.be < bes or s.p_eojeol > eojeol_fraction_s)

def fraction_of_jsubr_is_josa(l, jsub):
    rdict = {r:f for r, f in lrgraph.get(l+jsub, {}).items() if r != ''}
    f_sum = sum(rdict.values())
    if not f_sum:
        return 1.0 if jsub in josa_dict else 0.0
    f_jsubj = sum([f for r,f in rdict.items() if (jsub+r) in josa_dict])
    return (f_jsubj / f_sum)

def is_jsubr_josa(l, jsub, min_fraction=0.6):
    return fraction_of_jsubr_is_josa(l, jsub) > min_fraction

print(is_jsubr_josa('상태로서', '의'),'\n')

for w_, w in [('과학활동', '과학활동에'), ('지역주민', '지역주민과'), ('예술', '예술가'), ('상태', '상태로서')]:
    print(w_, w)
    pretty(score_dict[w_])
    pretty(score_dict[w])
    print('Nsub + J ? ', is_NsubJ(score_dict[w_], score_dict[w]))
    print('Jsub+R = J?', is_jsubr_josa(w_, w[len(w_):]), '%.3f'%fraction_of_jsubr_is_josa(w_, w[len(w_):]))
    print('N + Jsub ? ', is_NJsub(score_dict[w_], score_dict[w]), end='\n\n')

True 

과학활동 과학활동에
Score(score=3.564, frequency=34, be=1.655, av=7, p_feature=0.971, p_eojeol=0.000)
Score(score=6.485, frequency=12, be=-0.000, av=1, p_feature=1.000, p_eojeol=0.833)
Nsub + J ?  False
Jsub+R = J? True 1.000
N + Jsub ?  True

지역주민 지역주민과
Score(score=2.608, frequency=75, be=1.648, av=11, p_feature=0.576, p_eojeol=0.107)
Score(score=6.485, frequency=10, be=-0.000, av=1, p_feature=1.000, p_eojeol=0.900)
Nsub + J ?  False
Jsub+R = J? True 1.000
N + Jsub ?  True

예술 예술가
Score(score=2.589, frequency=4386, be=2.851, av=90, p_feature=0.673, p_eojeol=0.124)
Score(score=2.877, frequency=603, be=2.450, av=32, p_feature=0.792, p_eojeol=0.124)
Nsub + J ?  False
Jsub+R = J? False 0.436
N + Jsub ?  False

상태 상태로서
Score(score=2.457, frequency=4624, be=2.380, av=52, p_feature=0.991, p_eojeol=0.097)
Score(score=6.485, frequency=20, be=-0.000, av=1, p_feature=1.000, p_eojeol=0.700)
Nsub + J ?  False
Jsub+R = J? True 1.000
N + Jsub ?  True



In [23]:
candidates = dict(filter(lambda x:x[1].score > min_noun_score, score_dict.items()))
len(candidates)

57775

In [24]:
nouns = {}

for w, s in sorted(candidates.items(), key=lambda x:len(x[0])):
    n = len(w)
    if n == 1:
        nouns[w] = s
        continue
    # Case: 떡볶 + 이 --> remove 떡볶
    if (w[-1] in postnoun) and (w[:-1] in nouns):
        s_ = nouns[w[:-1]]
        if is_NsubJ(s_, s):
            del nouns[w[:-1]]
    
    # Case: [대학생 + 과] + 의 --> pass 대학생과 e = 1
    # Case: [정국 + 으로] + 의 --> pass 정국으로 e = 2
    NJsub = False
    for e in range(1, max(5, n)):
        l = w[:-e]
        jsub = w[-e:]
        if (l in nouns) and (jsub in josa_dict):
            s_ = nouns[l]
            if is_NJsub(s_, s):
                NJsub = True
                break
            if 0 == sum((1 if (jsub+r) in josa_dict else 0 for r in lrgraph.get(l, {}).keys())):
                NJsub = True
                break
    if NJsub:
        continue

    nouns[w] = s

len(candidates), len(nouns)

(57775, 50092)

In [25]:
words = ['예술가', '과거', '정국', '취재', '개체', '배후']
for w in words:
    print(w, w in nouns)

예술가 True
과거 True
정국 True
취재 True
개체 True
배후 True


In [26]:
'  '.join(tuple(zip(*sorted(nouns.items(), key=lambda x:x[1].score, reverse=True)[:300]))[0])

'여타  트윈  쌍마자동차  장상  노재헌  상호주  반나  경제정  사회체  랄프  반골  제야  토말  신상옥  최고인민회  화랭이패  베트남식  제반  중외  장단기  거국  혈중  자파  산업상  라오  순위항목  김중권  한국과학기술  다민족  라일  피개  가향  직업상  제가끔  샨  차렷  덕원  자와  지용  방신영  영화진흥  우리들끼리  무역주  불러라  무반  야누스  혼외  최현배  줄리어스  암묵  불굴  보라매  고야  구로구  도범  필요불가  공업사  경동맥동  안면근  히포  인조견  위자  김근태  자연생  김태정  시온  안암  인상주  루츠  덮어놓고  월하  구포  한국은행법  지역사  농공  인터컨티넨탈  이혜경  알과  방송문화  함북  지역환경  투자계  대륙간  행정사  부존  동맥류  뇌동맥  통합전  타종  간부급  지배주  한국기독교  한국미술  희대  알리안  개념상  핵확산금지조약  함경남도  마에다  국소  여일간  언어상  수개  앨리  한국산업  린다  신강  기획회  연쇄살인  지역주  저스틴  여분간  가계신용  각권  임원십육지  건설기  주구점  경남도  클린업  박광태  인민민주주  초현실  고부간  엘리아  탈자본주  태환  오륙십  워렌  신윤복  파죽  업무협  한낱  사회정  의료분쟁조정  신민주  마을공동  아홉번째  주간한국  푸나  완전식  나노  해마다  예의상  마샬  형사상  지오그래픽  고미  세계여성  근대민족  김현승  광통신  지방주  백민  환경주  뿔뿔이  요주  그들끼리  긴급회  아이스하키  토지분급  칠보  지금껏  외짝  활동계획  하루종일  민사상  태진아  김홍도  구운몽  우국  임금상  세브란스  인쇄공  학제간  열흘간  인간중심  아더  한국고대  문화주  반씩  뭐하고  노트르담  초감각  독일식  영어강  현대주  미증유  보슈  수간  통계자  가와  수도인  까자  장영주  김덕수  사회심리  영과  광진구  연구계  재판과정에서  정책자

### Compound: 소수 + [집단 + 의]

    소수 + 집단의 (집단의 = 집단 + 의 인지 확인)
    Noun score 대체


### Compound: 현대, 현대주

    '현대', '현대주', '주의', '주택' in nouns
    '현대주의', '현대주택' not in nouns
    
    현대: Score(score=3.778, frequency=5278, be=3.297, av=99, p_feature=0.342, p_eojeol=0.281),
    
    현대주:Score(score=6.243, frequency=10, be=0.610, av=2, p_feature=0.1, p_eojeol=0.0)
    
### Compound: 형사, 형사과 similar (현대, 현대주)

    '형사', '형사과', '과장', '과실'  in nouns
    '형사과장', '형사과실' not in nouns
    
    형사: Score(score=3.77, f=5278, be=3.29, av=99, p_feature=0.34, p_eojeol=0.28),
    형사과: Score(score=6.24, f=10, be=0.61, av=2, p_feature=0.1, p_eojeol=0.0))
    
    lrgraph['형사과']:
    {'': 1, '실의': 1, '의': 1, '장': 1, '장은': 3, '장의': 2, '장이': 2}

In [31]:
'형사' in nouns, '과장' in nouns, '과실' in nouns, '형사과' in nouns, '형사과장' in nouns, '형사과실' in nouns

(True, True, True, True, False, False)

In [27]:
'소수' in nouns, '민족' in nouns, '소수민' in nouns, '소수민족' in nouns

(True, True, False, True)

In [28]:
score_dict['소수'], score_dict['소수민족']

(Score(score=4.9341737325328783, frequency=642, be=2.2178874325972768, av=35, p_feature=0.6301075268817204, p_eojeol=0.19158878504672897),
 Score(score=2.8052688871142193, frequency=49, be=2.285407600563008, av=13, p_feature=0.8235294117647058, p_eojeol=0.2857142857142857))

In [29]:
score_dict['형사'], score_dict['형사과']

(Score(score=2.1159765739697289, frequency=857, be=2.9148398094956578, av=45, p_feature=0.5018518518518519, p_eojeol=0.23570595099183198),
 Score(score=6.4853445304586783, frequency=11, be=0.63903185965017695, av=3, p_feature=0.1, p_eojeol=0.09090909090909091))

### Extraction을 한 뒤, L-R graph 수정

'소수집단/명사 + 의'를 알고 난 뒤, [소수 - 집단\*]에 대한 링크를 모두 지워줘야 '소수'에 대한 판단이 명확해짐. repeatly extraction을 해야 함 